In [1]:
import sympy as sp

SUB_TO_DIGIT = {'₀':'0','₁':'1','₂':'2','₃':'3','₄':'4','₅':'5','₆':'6','₇':'7','₈':'8','₉':'9'}
DIGIT_TO_SUB = {v:k for k,v in SUB_TO_DIGIT.items()}

def parse_symbol_name(name):
    # underscore: r_12
    if '_' in name:
        prefix, idx = name.split('_', 1)
        if idx.isdigit():
            return prefix, idx, 'underscore'
    # ascii digits: r12
    i = len(name)-1
    while i >= 0 and name[i].isdigit():
        i -= 1
    if i < len(name)-1:
        return name[:i+1], name[i+1:], 'ascii'
    # unicode subscripts: r₁₂
    i = len(name)-1
    while i >= 0 and name[i] in SUB_TO_DIGIT:
        i -= 1
    if i < len(name)-1:
        prefix = name[:i+1]
        idx = ''.join(SUB_TO_DIGIT[ch] for ch in name[i+1:])
        return prefix, idx, 'unicode'
    return None, None, None

def build_name(prefix, new_index, style):
    if style == 'underscore':
        return f"{prefix}_{new_index}"
    if style == 'unicode':
        return f"{prefix}{''.join(DIGIT_TO_SUB[d] for d in str(new_index))}"
    return f"{prefix}{new_index}"

def shift_expr_variables(expr, shift):
    # Collect symbols robustly
    symbols = expr.atoms(sp.Symbol)
    if not symbols:
        return expr
    subs = {}
    for s in symbols:
        prefix, idx_str, style = parse_symbol_name(s.name)
        if prefix is None:
            continue
        new_idx = int(idx_str) + int(shift)
        new_name = build_name(prefix, new_idx, style)
        # Preserve assumptions
        subs[s] = sp.Symbol(new_name, **s.assumptions0)
    return expr.xreplace(subs)  # faster, exact structure

def shift_matrix_variables(matrix, shift):
    return matrix.applyfunc(lambda e: shift_expr_variables(e, shift))


r1, r2, r3, r4, r5 = sp.symbols('r1 r2 r3 r4 r5')
M = sp.Matrix([
    [(r1+r2+r3)/(r1*r2 + r1*r4 + r1*r5 + r2*r3 + r2*r5 + r3*r4 + r3*r5),
     r2/(r2*r4 + r2*r5 + r2*r3 + r3*r4 + r3*r5)],
    [(r1*r2 + r1*r4 + r1*r5 + r2*r3 + r2*r5 + r3*r4 + r3*r5)/r2,
     (r1*r2 + r1*r4 + r1*r5 + r2*r3 + r2*r5 + r3*r4 + r3*r5)/
     (r1*r2 + r1*r4 + r1*r5 + r2*r3 + r2*r5 + r3*r4 + r3*r5)]
])
M_shifted = shift_matrix_variables(M, 10)
M_shifted


Matrix([
[(r11 + r12 + r13)/(r11*r12 + r11*r14 + r11*r15 + r12*r13 + r12*r15 + r13*r14 + r13*r15), r12/(r12*r13 + r12*r14 + r12*r15 + r13*r14 + r13*r15)],
[              (r11*r12 + r11*r14 + r11*r15 + r12*r13 + r12*r15 + r13*r14 + r13*r15)/r12,                                                     1]])